# Notebook to demonstrate the Chebyshav-based likelihood fiting

First import few libraries
Note, that for the fitter itself, also the library `numba` and `iminuit` must be installed

In [ ]:
import sys

import numpy as np
import matplotlib.pyplot as plt

# To automaticaly reload when library-code is modified
%load_ext autoreload
%autoreload 2

import chebFitter as chf
import chebFitter2D as chf2D

## 1D fit

Generate the random Gaussian numbers between -1 and 1

In [ ]:
data = np.random.normal(loc=0.0, scale=1.0, size=10000)
data = data[ (-1 < data) & (data < 1)]


Define the Gaussian function (compiled for better performance)

In [ ]:
def myGauss(x, pars):
    from math import erf
    #return 0.5
    mean  = pars["mean"]
    sigma = pars["sigma"]
    xMin, xMax = -1, 1
    a = xMin - mean
    b = xMax - mean
    # the normalization to 1 is needed only for the slow fit option
    N =  np.sqrt(np.pi/2) * sigma* ( erf(b/np.sqrt(2)/sigma)  - erf(a/np.sqrt(2)/sigma) )
    f = 1./N * np.exp( -1./2 * pow((x-mean)/sigma, 2))
    assert(f >= 0)
    return f



In [ ]:
# check that it works
myGauss(1, {"mean": 0, "sigma" : 0.3})

Init and test the 1D Chebyshev fitter

In [ ]:
ch = chf.chebFitter(32 + 1, -1, 1, data, myGauss)

pars = {"mean" : 0, "sigma" : 1}

print('Slow likelihood is', ch.getLogLikelihoodSlow(pars))
print('Fast likelihood is', ch.getLogLikelihoodFast(pars))
print('Eval is', ch.eval(pars))

In [ ]:
pars = {"mean": 0, "sigma" : 1}
res =  ch.fitData(pars) # by default useCheb = True
print('Fast fitter result', res)

res =  ch.fitData(pars, useCheb = False) 
print('Slow fitter result', res)

Plot the histogram with the result

In [ ]:
plt.figure(figsize=(12,8))
plt.hist(data, histtype='step', density = True, bins=10)
x  = np.linspace(-1, 1, 100)
y  = np.vectorize(lambda a: myGauss(a, res))(x)
yCh= ch.funFast(x, res)
plt.plot(x, yCh)

## 2D fit

First generate the random distribution

In [ ]:
dataX = np.random.normal(loc=0.0, scale=1.0, size=1000000)
dataY = np.random.normal(loc=0.3, scale=0.7, size=1000000)
dataX = dataX[ (-1 < dataX) & (dataX < 1)]
dataY = dataY[ (-1 < dataY) & (dataY < 1)]
dataX.resize(min(len(dataX), len(dataY)))
dataY.resize(min(len(dataX), len(dataY)))
dataX.size, dataY.size
data2 = np.column_stack((dataX,dataY))

In [ ]:
def myGauss2D(x, pars):
    parX = { "mean"  : pars["meanX"],
            "sigma" : pars["sigmaX"] }
    parY = { "mean"  : pars["meanY"],
            "sigma" : pars["sigmaY"] }
    return myGauss(x[0], parX) * myGauss(x[1], parY)

In [ ]:
ch2 = chf2D.chebFitter(32 + 1, -1, 1, 32+1, -1, 1, data2, myGauss2D)

pars = {"meanX" : 0.0, "sigmaX" : 1.1,
        "meanY" : 0.3, "sigmaY" : 1.3}

print('Slow likelihood is', ch2.getLogLikelihoodSlow(pars))
print('Fast likelihood is', ch2.getLogLikelihoodFast(pars))
print('Eval is', ch2.eval(pars))

In [ ]:
res2 =  ch2.fitData(pars) # By default is chebFit
print('Fast fitter result', res2)

#res2 =  ch2.fitData(pars, useCheb = False) 
#print('Slow fitter result', res2)

In [ ]:
plt.figure(figsize=(12,8))
plt.hist(data2[:,0], histtype='step', density = True, bins=40)
x  = np.linspace(-1, 1, 100)
yCh= ch2.funFastProjX(x, res2)
plt.plot(x, yCh)